### 1단계 - LLM 양자화에 필요한 패키지 설치
bitsandbytes: Bitsandbytes는 CUDA 사용자 정의 함수, 특히 8비트 최적화 프로그램, 행렬 곱셈(LLM.int8()) 및 양자화 함수에 대한 경량 래퍼
PEFT(Parameter-Efficient Fine-Tuning): 모델의 모든 매개변수를 미세 조정하지 않고도 사전 훈련된 PLM(언어 모델)을 다양한 다운스트림 애플리케이션에 효율적으로 적용 가능
accelerate: PyTorch 모델을 더 쉽게 여러 컴퓨터나 GPU에서 사용할 수 있게 해주는 도구

In [1]:
#양자화에 필요한 패키지 설치
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

### 2단계 - 트랜스포머에서 BitsandBytesConfig를 통해 양자화 매개변수 정의하기
load_in_4bit=True: 모델을 4비트 정밀도로 변환하고 로드하도록 지정
bnb_4bit_use_double_quant=True: 메모리 효율을 높이기 위해 중첩 양자화를 사용하여 추론 및 학습
bnd_4bit_quant_type="nf4": 4비트 통합에는 2가지 양자화 유형인 FP4와 NF4가 제공됨. NF4 dtype은 Normal Float 4를 나타내며 QLoRA 백서에 소개되어 있습니다. 기본적으로 FP4 양자화 사용
bnb_4bit_compute_dype=torch.bfloat16: 계산 중 사용할 dtype을 변경하는 데 사용되는 계산 dtype. 기본적으로 계산 dtype은 float32로 설정되어 있지만 계산 속도를 높이기 위해 bf16으로 설정 가능

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

/home/LLM/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 3단계 - 경량화 모델 로드하기
이제 모델 ID를 지정한 다음 이전에 정의한 양자화 구성으로 로드합니다.

In [3]:
model_id = "kyujinpy/Ko-PlatYi-6B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.66s/it]


### 4단계 - 잘 실행되는지 확인

In [6]:
device = "cuda:0"

messages = [
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해줘"}
]

# Define a simple chat template
chat_template = "{% for message in messages %}{% if message.role == 'user' %}### User:\n{{ message.content }}\n{% elif message.role == 'assistant' %}### Assistant:\n{{ message.content }}\n{% endif %}{% endfor %}"

# Apply the chat template
encodeds = tokenizer.apply_chat_template(messages, chat_template=chat_template, return_tensors="pt")

model_inputs = encodeds.to(device)


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

### 5단계- RAG 시스템 결합하기

In [7]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [10]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """

koplatyi_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)

/home/LLM/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/home/LLM/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [15]:
loader = PyPDFLoader("/home/LLM/intel.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(texts, hf)
retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={'k': 3}
                        )

In [16]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
result = rag_chain.invoke("4월에는 무엇을 배웠지?")

for i in result['context']:
    print(f"주어진 근거: {i.page_content} / 출처: {i.metadata['source']} - {i.metadata['page']} \n\n")

print(f"\n답변: {result['text']}")

주어진 근거: 4 월 10 일 - 공강
4 월 11 일 - Pandas 소개, Pandas 코드 실습
4 월 12 일 - Matplotlib, Scikit-learn 소개, Matplotlib, Scikit-learn 코드 실습
4 월 15 일 - 데이터  가져오기  및 처리, Data Import and Processing 실습, Seq_Coding 실습
4 월 16 일 -  Basic Data Processing and Visualization 실습, Handling erroneous and 
missing data 실습, 팀 프로젝트  진행
4 월 17 일 - 통계 데이터  연습을  위한 AI, Machine Learning Techniques 실습
4 월 18 일 - Regression 실습 
4 월 19 일 - Class 와 pygae 을 이용한  간단한  게임제작 , 캡스톤  팀 프로젝트 , Rossmann 
매장 데이터를  이용한  Sales 예측 프로젝트 / 출처: /home/LLM/intel.pdf - 1 


주어진 근거: 4 월 2 일 - 새로운  기술 - AI, 쉬운 AI, AI 안에 무엇이  있는가 ?, AI 받아들이기 , 포용적  AI, AI 
목표설정 
4 월 3 일 - AI 프로젝트  사이클  소개, 4W 소개, 데이터  소스, 데이터  탐색, 모델링 , 평가, 
배포, 지도학습 , 비지도학습 , No 코드 소프트웨어인  Orange 를 이요한  AI 작업(Employee 
Attrition Prediction, Predictive Maintenance) 
4 월 4 일 - No 코드 소프트웨어인  Orange 를 이용한  AI 작업 : Insurance Fraud 
Detection,No 코드 소프트웨어인  Orange 를 이용한  AI 작업 : Quality Assurance System, 
No 코드 소프트웨어인  Orange 를 이용한  AI 작업 : Viral Post Prediction, No 코드 / 출처: /home/